<a href="https://colab.research.google.com/github/calvinli08/tf-demo/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras import layers

import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

#get dataset and format
(train_ds, validate_ds, test_ds), ds_info = tfds.load(
    'HorsesOrHumans',
    split=['train[:90%]', 'train[90%:100%]', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
    batch_size=32
)

#normalize data so the model trains more efficiently
def normalize(image, label):
  image = tf.cast(image/255, tf.float32)
  return image, label

train_ds = train_ds.map(normalize)
validate_ds = validate_ds.map(normalize)
test_ds = test_ds.map(normalize)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incomplete642J5P/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incomplete642J5P/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [2]:
#prepare model layers
model = keras.Sequential([
    #randomize the training data to increase data diversity
    layers.RandomFlip("horizontal_and_vertical"),

    #use convolution to highlight the parts of the image useful for identification
    #kernel regularizer should help prevent overfitting
    layers.Conv2D(
        16,
        (3,3),
        activation='relu',
        input_shape=(300, 300, 3),
        kernel_regularizer=regularizers.l2(0.01)
    ),

    #cut down on excess image info returned from the convolution layer and keep just the essentials
    layers.MaxPooling2D(2,2),

    #second convolution layer
    #gradually increase output channels as model hones in on image details
    layers.Conv2D(
        32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)
    ),

    #cut down on excess image info returned from the convolution layer and keep just the essentials
    layers.MaxPooling2D(2,2),

    #third convolution layer
    #gradually increase output channels as model hones in on image details
    layers.Conv2D(
        64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)
    ),

    #cut down on excess image info returned from the convolution layer and keep just the essentials
    layers.MaxPooling2D(2,2),

    #prepare to feed to dense layer
    layers.Flatten(),

    #classify using dense layers
    layers.Dense(
        64, activation='relu', kernel_regularizer=regularizers.l2(0.01)
    ),

    layers.Dropout(0.5),

    layers.Dense(2)
])

#set model optimizer and loss equation
model.compile(
    #Using adam since it's quickest and most accurate for basic data sets
    optimizer='adam',

    #Since we have two categories, human and horse, SparseCategoricalCrossentropy should be most effective
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
      'accuracy'
    ]
)

model.build((None,300,300,3))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 300, 300, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        1

In [4]:
#train and test (using just 5 epochs to save resources as it's just a demo)
checkpoint_filepath = 'quick_demo.cpkt'
save_cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    verbose=1
)

early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=200
)

model.fit(
    train_ds,
    epochs=5,
    validation_data=validate_ds,
    batch_size=32,
    callbacks=[
        save_cp_callback,
        early_stopping_callback
    ]
)

loss, acc = model.evaluate(test_ds, verbose=2)

print("\nTest results - Loss:", "{:.2f}".format(loss), "Accuracy:", "{:.2f}".format(acc))

Epoch 1/5
29/29 [==============================] - ETA: 0s - loss: 0.6095 - accuracy: 0.9383
Epoch 1: saving model to quick_demo.cpkt
29/29 [==============================] - 4s 143ms/step - loss: 0.6095 - accuracy: 0.9383 - val_loss: 0.6276 - val_accuracy: 0.9417
Epoch 2/5
29/29 [==============================] - ETA: 0s - loss: 0.6651 - accuracy: 0.9102
Epoch 2: saving model to quick_demo.cpkt
29/29 [==============================] - 4s 145ms/step - loss: 0.6651 - accuracy: 0.9102 - val_loss: 0.5905 - val_accuracy: 0.9417
Epoch 3/5
29/29 [==============================] - ETA: 0s - loss: 0.5855 - accuracy: 0.9361
Epoch 3: saving model to quick_demo.cpkt
29/29 [==============================] - 5s 159ms/step - loss: 0.5855 - accuracy: 0.9361 - val_loss: 0.5586 - val_accuracy: 0.9515
Epoch 4/5
29/29 [==============================] - ETA: 0s - loss: 0.5199 - accuracy: 0.9610
Epoch 4: saving model to quick_demo.cpkt
29/29 [==============================] - 4s 144ms/step - loss: 0.5199 -